In [2]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
# import rs_funcs as rs
import

In [3]:
ds = xr.open_dataset('redsun_timeseries_17_10.nc')
sg = xr.open_dataset('StupidGridFull.nc', group='flux')

In [4]:
def ls2sol(ls):
    N_s = 668.6
    ls_peri = 250.99
    t_peri = 485.35
    a = 1.52368
    e = 0.09340
    epsilon = 25.1919
    if (ls == 0).any():
        ls = .01
    nu = np.radians(ls) + 1.90258
    E = np.arctan((np.tan(nu/2))/(np.sqrt((1+e)/(1-e))))*2
    M = E - e*np.sin(E)
    Ds = (M/(2*np.pi))*N_s + t_peri
    if (Ds < 0).any():
        Ds = Ds + N_s
    if (Ds > N_s).any():
        Ds = Ds - N_s
    return(Ds)


In [5]:
lat = ds['lat'][0]
lon = ds['lon'][0]
G = np.zeros(len(ds['lon']))
for ri in range(0,len(ds['lon'])):
    ls = ds['ls'][ri]
    hr = ds['hr'][ri]
    G[ri] = sg['flux_dw_sw'][lat,lon,ls,hr]
lss = np.unique(ds['ls'])

In [15]:
# 1j-pv
# for testing
P = np.array(G[:, np.newaxis] * ds['j1_etaPV'])
zz = np.zeros((len(lss),len(ds['j1-bg'])))
for i in range(0,len(lss)):
    hr_int = np.where(ds['ls']==lss[i])
    x = np.array(ds['hr'][hr_int] * 2 * 1.02569 * 60 * 60)
    for j in range(0,len(ds['j1-bg'])):
        y = P[:,j][hr_int]
        z = np.trapz(y,x=x)
        zz[i,j] = z
z = np.zeros((len(ds['j1-bg'])))
x = ls2sol(lss) * 1.02569 * 24 * 60 * 60
for j in range(0,len(ds['j1-bg'])):
    y = zz[:,j]
    z[j] = np.trapz(y,x=lss)
j1pv = np.max(z)
j1pvi = np.argmax(z)

In [21]:
j1pv * (1/688) * (1/1000) * (1/3600)

3.7744996696842144

In [13]:
# 2j-aa
Z = 60.052
n = 8
F = 96485.33212
P = G[:, np.newaxis, np.newaxis] * ds['j2_etaPEC_AA_2bg'] * Z/(n*F)
zz = np.zeros((len(lss),len(ds['j2-bg1']),len(ds['j2-bg2'])))
for i in range(0,len(lss)):
    hr_int = np.where(ds['ls']==lss[i])
    x = np.array(ds['hr'][hr_int] * 2 * 1.02569 * (1/60) * (1/60))
    for j in range(0,len(ds['j2-bg1'])):
        for k in range(0,len(ds['j2-bg2'])):
            y = P[:,j,k][hr_int]
            z = np.trapz(y,x=x)
            zz[i,j,k] = z
z = np.zeros((len(ds['j2-bg1']),len(ds['j2-bg2'])))
x = ls2sol(lss)
for j in range(0,len(ds['j2-bg1'])):
    for k in range(0,len(ds['j2-bg2'])):
        y = zz[:,j,k]
        z[j,k] = np.trapz(y,x=lss)
j2aa = np.max(z)
j2aai = np.unravel_index(np.argmax(z),np.shape(z), order='C')


In [22]:
# 2j-h2
Z = 2.016
n = 2
F = 96485.33212
P = G[:, np.newaxis, np.newaxis] * ds['j2_etaPEC_H2_2bg'] * Z/(n*F)
zz = np.zeros((len(lss),len(ds['j2-bg1']),len(ds['j2-bg2'])))
for i in range(0,len(lss)):
    hr_int = np.where(ds['ls']==lss[i])
    x = np.array(ds['hr'][hr_int] * 2 * 1.02569 * 60 * 60)
    for j in range(0,len(ds['j2-bg1'])):
        for k in range(0,len(ds['j2-bg2'])):
            y = P[:,j,k][hr_int]
            z = np.trapz(y,x=x)
            zz[i,j,k] = z
z = np.zeros((len(ds['j2-bg1']),len(ds['j2-bg2'])))
x = ls2sol(lss) * 1.02569 * 24 * 60 * 60
for j in range(0,len(ds['j2-bg1'])):
    for k in range(0,len(ds['j2-bg2'])):
        y = zz[:,j,k]
        z[j,k] = np.trapz(y,x=lss)
j2h2 = np.max(z)
j2h2i = np.unravel_index(np.argmax(z),np.shape(z), order='C')

In [26]:
# j2h2
j2h2 * 1/688

123.42632743292856

In [ ]:
# 2j-h3
Z = 17.031
n = 6
F = 96485.33212
P = G[:, np.newaxis, np.newaxis] * ds['j2_etaPEC_H2_2bg'] * Z/(n*F)
zz = np.zeros((len(lss),len(ds['j2-bg1']),len(ds['j2-bg2'])))
for i in range(0,len(lss)):
    hr_int = np.where(ds['ls']==lss[i])
    x = np.array(ds['hr'][hr_int] * 2 * 1.02569 * (1/60) * (1/60))
    for j in range(0,len(ds['j2-bg1'])):
        for k in range(0,len(ds['j2-bg2'])):
            y = P[:,j,k][hr_int]
            z = np.trapz(y,x=x)
            zz[i,j,k] = z
z = np.zeros((len(ds['j2-bg1']),len(ds['j2-bg2'])))
x = ls2sol(lss)
for j in range(0,len(ds['j2-bg1'])):
    for k in range(0,len(ds['j2-bg2'])):
        y = zz[:,j,k]
        z[j,k] = np.trapz(y,x=lss)
j2h2 = np.max(z)
j2h2i = np.unravel_index(np.argmax(z),np.shape(z), order='C')

44

In [ ]:
# 3j-pv
P = G[:, np.newaxis, np.newaxis, np.newaxis] * ds['j3_etaPV_3bg']
zz = np.zeros((len(lss),len(ds['j3-bg1']),len(ds['j3-bg2']),len(ds['j3-bg3'])))
for i in range(0,len(lss)):
    hr_int = np.where(ds['ls']==lss[i])
    x = np.array(ds['hr'][hr_int] * 2 * 1.02569 * (1/60) * (1/60))
    for j in range(0,len(ds['j3-bg1'])):
        for k in range(0,len(ds['j3-bg2'])):
            for l in range(0,len(ds['j3-bg3'])):
                y = P[:,j,k,l][hr_int]
                z = np.trapz(y,x=x)
                zz[i,j,k,l] = z
z = np.zeros((len(ds['j3-bg1']),len(ds['j3-bg2']),len(ds['j3-bg3'])))
x = ls2sol(lss)
for j in range(0,len(ds['j3-bg1'])):
    for k in range(0,len(ds['j3-bg2'])):
        for l in range(0,len(ds['j3-bg3'])):
            y = zz[:,j,k,l]
            z[j,k,l] = np.trapz(y,x=lss)
j3pv = np.max(z)
j3pvi = np.unravel_index(np.argmax(z),np.shape(z), order='C')


In [ ]:
j1pvF = np.zeros((19,37))

In [ ]:
t = xr.DataArray(j1pvF)

In [ ]:
t.to_netcdf('test')

In [ ]:
t.to_netcdf()

t

In [ ]:
t.to_dataframe('testframe')

In [ ]:

plt.contour(t.to_pandas())

In [33]:

lats = np.arange(-90,91,10)
lons = np.arange(-180,181,10)


j1pvF = np.zeros((19,37))
j1pviF = np.zeros((19,37))

test = xr.Dataset(
    {'j1pvF': (('lon','lat'), j1pvF.transpose()),
    'j1pviF': (('lon','lat'), j1pviF.transpose())},
    coords = {
        'lon': lons,
        'lat': lats,
    },
)

test['lat'].attrs['units'] = 'degrees_north'
test['lon'].attrs['units'] = 'degrees_east'
test['j1pvF'].attrs['units'] = 'J/m^2'
test['j1pviF'].attrs['units'] = 'nm'

test['j1pvF'] = test['j1pvF'].where(test['j1pvF'] != 0.0)
test['j1pviF'] = test['j1pviF'].where(test['j1pviF'] != 0.0)

test.to_netcdf('testnet.nc')



In [34]:
test['j1pvF'] = test['j1pvF'].where(test['j1pvF'] != 0.0)

In [35]:
test

<xarray.Dataset>
Dimensions:  (lat: 19, lon: 37)
Coordinates:
  * lon      (lon) int64 -180 -170 -160 -150 -140 -130 ... 140 150 160 170 180
  * lat      (lat) int64 -90 -80 -70 -60 -50 -40 -30 ... 30 40 50 60 70 80 90
Data variables:
    j1pvF    (lon, lat) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    j1pviF   (lon, lat) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [ ]:
5t[1,3]

array([[0.00000000e+000, 2.14269753e-314, 0.00000000e+000,
        1.15117295e-321, 4.44659081e-323, 0.00000000e+000,
        6.95017380e-310, 6.99670457e+253, 6.95018612e-310,
        2.54337914e-288, 6.95017379e-310, 3.92842582e+153,
        6.95015993e-310, 2.39079755e-222, 6.95014635e-310,
        1.45060229e-137, 6.95021261e-310, 5.87339298e-283,
        6.95017380e-310, 1.38905875e-002, 6.95018681e-310,
        2.51951619e+093, 6.95014630e-310, 1.72611514e-283,
        6.95021261e-310, 4.42057230e+223, 6.95016020e-310,
        1.35459110e-040, 6.95021261e-310, 4.02379893e+237,
        6.95021261e-310, 2.15423221e-216, 6.95021261e-310,
        2.90207736e-119, 6.95021261e-310, 3.43519746e+073,
        6.95021262e-310],
       [3.95252517e-323, 6.95021261e-310, 3.83157261e-276,
        6.95021261e-310, 4.03920841e+231, 6.95021261e-310,
        5.91430289e+116, 6.95021272e-310, 6.67571334e+112,
        6.95021261e-310, 1.63636212e+047, 6.95021263e-310,
        4.37329485e+262, 6.950